In [1]:
pip install openrouteservice


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
import openrouteservice as ors
import csv
from openrouteservice.exceptions import ApiError

In [39]:
API_key = '5b3ce3597851110001cf62483c393d0b692343f19e36d03cb861f456'

In [40]:
client = ors.Client(key=API_key)

In [4]:
repo_url='https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
vaud_cities = pd.read_excel(repo_url+'vaud_communes.xlsx')
display(vaud_cities)

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,91 ans,92 ans,93 ans,94 ans,95 ans,96 ans,97 ans,98 ans,99 ans,100+ ans
0,Aigle,1860,Aigle,5401,VD,2563821.020,1129273.277,fr,11780,138,...,20,25,16,6,9,7,0,3,5,4
1,Le Sépey,1863,Aigle,5401,VD,2568544.893,1131955.818,fr,11780,138,...,20,25,16,6,9,7,0,3,5,4
2,Bex,1880,Bex,5402,VD,2567230.358,1121779.532,fr,8806,79,...,14,11,13,14,8,6,2,0,1,3
3,Frenières-sur-Bex,1880,Bex,5402,VD,2571040.657,1121183.948,fr,8806,79,...,14,11,13,14,8,6,2,0,1,3
4,Fenalet-sur-Bex,1880,Bex,5402,VD,2568662.358,1125041.373,fr,8806,79,...,14,11,13,14,8,6,2,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Villars-Epeney,1404,Villars-Epeney,5935,VD,2543412.650,1181535.124,fr,108,1,...,0,0,0,0,0,0,0,0,0,0
541,Vugelles-La Mothe,1431,Vugelles-La Mothe,5937,VD,2534182.487,1186399.984,fr,147,3,...,0,0,0,0,0,0,0,0,0,0
542,Yverdon-les-Bains,1400,Yverdon-les-Bains,5938,VD,2539180.581,1180801.624,fr,30332,245,...,69,70,50,44,24,18,17,4,7,6
543,Gressy,1432,Yverdon-les-Bains,5938,VD,2538768.002,1178103.684,fr,30332,245,...,69,70,50,44,24,18,17,4,7,6


In [5]:
coordonnees_venoge = [46.60396069250175, 6.538034286509177] #lat,long

In [6]:
import math
# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

        # Convert CH y/x to WGS long
def CHtoWGSlng( y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
        + (0.791484 * y_aux * x_aux) + \
        + (0.1306 * y_aux * pow(x_aux, 2))) + \
        - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng

In [7]:
vaud_cities['E_lv03'] = vaud_cities['E']-2000000
vaud_cities['N_lv03'] = vaud_cities['N']-1000000 

vaud_cities['lat'] = vaud_cities.apply(lambda x: CHtoWGSlat(x['E_lv03'], x['N_lv03']), axis=1)
vaud_cities['lng'] = vaud_cities.apply(lambda x: CHtoWGSlng(x['E_lv03'], x['N_lv03']), axis=1)


In [8]:
vaud_cities

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,95 ans,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng
0,Aigle,1860,Aigle,5401,VD,2563821.020,1129273.277,fr,11780,138,...,9,7,0,3,5,4,563821.020,129273.277,46.313885,6.968954
1,Le Sépey,1863,Aigle,5401,VD,2568544.893,1131955.818,fr,11780,138,...,9,7,0,3,5,4,568544.893,131955.818,46.338253,7.030098
2,Bex,1880,Bex,5402,VD,2567230.358,1121779.532,fr,8806,79,...,8,6,2,0,1,3,567230.358,121779.532,46.246650,7.013740
3,Frenières-sur-Bex,1880,Bex,5402,VD,2571040.657,1121183.948,fr,8806,79,...,8,6,2,0,1,3,571040.657,121183.948,46.241467,7.063181
4,Fenalet-sur-Bex,1880,Bex,5402,VD,2568662.358,1125041.373,fr,8806,79,...,8,6,2,0,1,3,568662.358,125041.373,46.276060,7.032088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Villars-Epeney,1404,Villars-Epeney,5935,VD,2543412.650,1181535.124,fr,108,1,...,0,0,0,0,0,0,543412.650,181535.124,46.782581,6.697615
541,Vugelles-La Mothe,1431,Vugelles-La Mothe,5937,VD,2534182.487,1186399.984,fr,147,3,...,0,0,0,0,0,0,534182.487,186399.984,46.825492,6.576052
542,Yverdon-les-Bains,1400,Yverdon-les-Bains,5938,VD,2539180.581,1180801.624,fr,30332,245,...,24,18,17,4,7,6,539180.581,180801.624,46.775610,6.642296
543,Gressy,1432,Yverdon-les-Bains,5938,VD,2538768.002,1178103.684,fr,30332,245,...,24,18,17,4,7,6,538768.002,178103.684,46.751304,6.637255


In [9]:
vaud_cities['distance_to_venoge_km'] = None

In [ ]:
route = client.directions(
    profile='driving-car',
    format='geojson',
    coordinates=[[6.968953, 46.313884],[6.5367833784414175, 46.60314203414339]] #longitude, latitude
)
print(route)
print(route['features'][0]['properties']['segments'][0]['distance'])

{'type': 'FeatureCollection', 'bbox': [6.536397, 46.313696, 6.968664, 46.605128], 'features': [{'bbox': [6.536397, 46.313696, 6.968664, 46.605128], 'type': 'Feature', 'properties': {'segments': [{'distance': 56227.8, 'duration': 2434.0, 'steps': [{'distance': 260.6, 'duration': 62.5, 'type': 11, 'instruction': 'Head northwest on Rue de la Chapelle', 'name': 'Rue de la Chapelle', 'way_points': [0, 8]}, {'distance': 23.7, 'duration': 1.9, 'type': 0, 'instruction': 'Turn left onto Avenue de Loës', 'name': 'Avenue de Loës', 'way_points': [8, 9]}, {'distance': 39.2, 'duration': 9.4, 'type': 13, 'instruction': 'Keep right onto Rue Margencel', 'name': 'Rue Margencel', 'way_points': [9, 11]}, {'distance': 669.9, 'duration': 74.8, 'type': 1, 'instruction': 'Turn right onto Rue Margencel', 'name': 'Rue Margencel', 'way_points': [11, 31]}, {'distance': 458.8, 'duration': 48.1, 'type': 7, 'instruction': "Enter the roundabout and take the 3rd exit onto Route d'Evian", 'name': "Route d'Evian", 'exit

In [11]:
vaud_cities.iloc[5]

Commune                         Les Plans-sur-Bex
NPA                                          1880
Commune d'annonce / District                  Bex
Commune d'annonce référence                  5402
Canton                                         VD
                                      ...        
E_lv03                                  574679.26
N_lv03                                 120863.945
lat                                     46.238735
lng                                      7.110372
distance_to_venoge_km                        None
Name: 5, Length: 115, dtype: object

In [ ]:
# Calculate distance to Venoge for each city
''' 
for index, row in vaud_cities.iterrows():
    city=f"{row['lng']},{row['lat']}"
    print(row['Commune'])
    try:
        route = client.directions(
            profile='driving-car',
            format='geojson',
            coordinates=[
                [coordonnees_venoge[1], coordonnees_venoge[0]],  # [lng, lat]
                [row['lng'], row['lat']]
            ]
        )

        distance_km = route['features'][0]['properties']['segments'][0]['distance'] / 1000
        vaud_cities.at[index, 'distance_to_venoge_km'] = round(distance_km, 2)

    except ApiError as e:
        print(f"API error for {row['Commune']}: {e}")
        vaud_cities.at[index, 'distance_to_venoge_km'] = None

    except Exception as e:
        print(f"Unexpected error for {row['Commune']}: {e}")
        vaud_cities.at[index, 'distance_to_venoge_km'] = None
'''

Aigle
Le Sépey
Bex
Frenières-sur-Bex
Fenalet-sur-Bex
Les Plans-sur-Bex
API error for Les Plans-sur-Bex: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 7.1103723 46.2387355.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1746509073984}})
Les Posses-sur-Bex
Gryon
Chessel
Corbeyrier
API error for Corbeyrier: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 6.9869118 46.3705863.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1746509074226}})
Frenières-sur-Bex
Les Posses-sur-Bex
Gryon
Lavey-Village
API error for Lavey-Village: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 7.0335064 46.2208585.'}, 'info': {'

/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Use

Bougy-Villars
Montherod
Pizy
Ballens
Berolle
Bière
Berolle
Bougy-Villars
Perroy
Féchy
Gimel
Longirod
Marchissy
Mollens VD
St-George
Bière
St-Livres
St-Oyens
Saubraz
Domdidier
Avenches
Oleyres
Donatyre
Misery
Cudrefin
Faoug
Villars-le-Grand
Salavaux
Bellerive VD
Cotterd
Vallamand
Montmagny
Constantine
Chabrey


/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 10th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Mur (Vully) VD
Bettens
Daillens
Bournens
Boussens
Chavannes-le-Veyron
La Chaux (Cossonay)
Chevilly
Chavannes-le-Veyron
Chevilly
Cossonay-Ville
Allens
Cuarnens
Daillens
Dizy
Eclépens
Ferreyres
Gollion
Grancy
L'Isle
Villars-Bozon
La Coudre
Lussery-Villars
Mauraz
Mex VD
Moiry VD
Mont-la-Ville
Vaulion
Montricher
Orny
Bavois
Penthalaz
Penthaz
Pompaples
La Sarraz
Senarclens
Sullens
Mex VD
Vufflens-la-Ville
Assens
Bioley-Orjulaz
Bercher
Bottens
Bretigny-sur-Morrens
Cugy VD
Echallens
Essertines-sur-Yverdon
Epautheyres
Etagnières
Fey
Froideville
Morrens VD
Oulens-sous-Echallens
Pailly
Penthéréaz
Poliez-Pittet
Rueyres
St-Barthélemy VD
Villars-le-Terroir
Vuarrens
Dommartin
Naz
Poliez-le-Grand
Sugnens
Goumoens-la-Ville
Eclagnens
Goumoens-le-Jux
Fontanezier
Bonvillars
Les Rasses
Bullet
Champagne
Concise
Vaumarcus
Corcelles-près-Concise
Fiez
Bullet
API error for Bullet: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 

In [13]:
vaud_cities.sort_values(by='distance_to_venoge_km', ascending=True, inplace=True)
vaud_cities.reset_index(drop=True, inplace=True)
vaud_cities

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,96 ans,97 ans,98 ans,99 ans,100+ ans,E_lv03,N_lv03,lat,lng,distance_to_venoge_km
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,2,1,0,1,0,531186.492,161298.778,46.599406,6.540556,0.74
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,3,1,1,3,0,530510.884,162919.512,46.613915,6.531495,2.72
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,0,0,0,0,0,533025.028,162344.417,46.608998,6.564397,3.51
3,Vufflens-la-Ville,1302,Vufflens-la-Ville,5503,VD,2530896.906,1158606.892,fr,1342,7,...,1,0,0,0,0,530896.906,158606.892,46.575163,6.537181,3.65
4,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,0,0,0,0,0,531837.469,164336.340,46.626796,6.548602,3.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Bullet,1453,Fiez,5556,VD,2532166.732,1190703.476,fr,431,1,...,0,0,0,0,0,532166.732,190703.476,46.863998,6.548997,None
541,St-Cergue,1264,Arzier-Le Muids,5702,VD,2499491.498,1147230.149,fr,2968,11,...,0,0,0,0,0,499491.498,147230.149,46.468854,6.130034,None
542,Gingins,1276,Chéserex,5709,VD,2497074.208,1142418.992,fr,1277,8,...,1,0,0,0,0,497074.208,142418.992,46.425214,6.099636,None
543,L'Etivaz,1660,Château-d'Oex,5841,VD,2578741.167,1139682.796,fr,3656,36,...,4,0,1,1,0,578741.167,139682.796,46.408162,7.162172,None


In [14]:
vaud_cities.to_csv('vaud_cities.csv', index=False)

In [20]:
vaud_cities[vaud_cities['distance_to_venoge_km'].isna()]
vaud_cities.dropna(subset=['distance_to_venoge_km'], inplace=True)

In [ ]:
# Filter cities within 5 to 15 km from Venoge
cities=vaud_cities[(vaud_cities['distance_to_venoge_km']>=5) & (vaud_cities['distance_to_venoge_km']<=15)]

In [ ]:
#cities=vaud_cities[:5]

In [ ]:
# coordinates = cities[['lng', 'lat']].values.tolist()
# n = len(coordinates)

# # Step 2: Initialize empty matrix
# distance_matrix = []

# # Step 3: Fill the matrix row by row
# for i in range(n):
#     row_distances = []
#     for j in range(n):
#         if i == j:
#             row_distances.append(0.0)
#             continue
#         try:
#             response = client.directions(
#                 profile='driving-car',
#                 format='geojson',
#                 coordinates=[coordinates[i], coordinates[j]]
#             )
#             distance_km = response['features'][0]['properties']['segments'][0]['distance'] / 1000.0
#             row_distances.append(round(distance_km))  
#         except ApiError as e:
#             print(f"API error from point {i} to {j}: {e}")
#             row_distances.append(None)
#         except Exception as e:
#             print(f"Unexpected error from point {i} to {j}: {e}")
#             row_distances.append(None)
#     distance_matrix.append(row_distances)

/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Use

API error from point 2 to 3: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 4: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 5: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 6: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 7: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 8: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 9: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 10: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 11: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 12: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 13: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 14: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 15: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 16: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 17: 403 ({'error': 'Quota exceeded'})
API error from point 2 to 18: 403 ({'error': 'Q

KeyboardInterrupt: 

In [ ]:
#Initialize empty matrix
'''distance_matrix = [[None] * n for _ in range(n)]

# Save the empty matrix to a CSV file
with open('distance_matrix.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(distance_matrix)'''

In [41]:
# Initialize a cache to store distances
distance_cache = {}

with open('distance_matrix.csv', 'r') as f:
    reader = csv.reader(f)
    distance_matrix = [[float(cell) if cell else None for cell in row] for row in reader]


In [43]:
# Step 3: Fill the matrix row by row
for i in range(n):
    for j in range(i, n):  # Only calculate for j >= i
        if distance_matrix[i][j] is None:
            if i == j:
                distance_matrix[i][j] = 0.0
            else:
                # Check if the distance is already in the cache
                coord_pair = tuple(sorted([tuple(coordinates[i]), tuple(coordinates[j])]))
                if coord_pair in distance_cache:
                    distance_km = distance_cache[coord_pair]
                else:
                    try:
                        response = client.directions(
                            profile='driving-car',
                            format='geojson',
                            coordinates=[coordinates[i], coordinates[j]]
                        )
                        distance_km = response['features'][0]['properties']['segments'][0]['distance'] / 1000.0
                        distance_cache[coord_pair] = round(distance_km)  # Cache the result
                    except ApiError as e:
                        print(f"API error from point {i} to {j}: {e}")
                        distance_km = None
                    except Exception as e:
                        print(f"Unexpected error from point {i} to {j}: {e}")
                        distance_km = None

                distance_matrix[i][j] = distance_km
                distance_matrix[j][i] = distance_km  # Reuse for the symmetric pair

/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zeliadecaillet/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Use

In [44]:
for row in distance_matrix:
    print(row)

[0.0, 6.8786000000000005, 8.2869, 9.662700000000001, 7.228899999999999, 5.1369, 10.9772, 10.1884, 10.3668, 12.1769, 7.074199999999999, 6.7338000000000005, 4.420100000000001, 13.200700000000001, 7.315600000000001, 10.4486, 11.2505, 8.317, 3.9298, 14.4357, 13.0336, 13.211799999999998, 11.6344, 8.931799999999999, 8.0894, 13.5338, 6.318899999999999, 7.6721, 11.704, 13.4114, 16.1421, 14.5003, 13.1111, 14.5561, 6.1349, 6.279100000000001, 7.2833000000000006, 6.3353, 9.6096, 9.7375, 9.9291, 6.8693, 14.5033, 10.1446, 13.2919, 7.1395, 7.7999, 15.9655, 8.6228, 13.9427, 7.8445, 18.0183, 9.0487, 11.2971, 15.2582, 16.7833, 16.811700000000002, 15.5257, 14.936399999999999, 11.8122, 9.784600000000001, 11.9922, 16.3768, 16.0138, 12.1199, 10.2264, 12.9779, 21.1256, 17.1088, 8.7285, 13.0739, 16.3865, 16.4306, 13.639, 9.5927, 10.245299999999999, 13.581700000000001, 16.8327]
[6.8786000000000005, 0.0, 9.231200000000001, 9.459200000000001, 1.4415, 2.3433, 5.9291, 4.401, 11.311, 11.9384, 11.963799999999999, 3.

In [45]:
with open('distance_matrix.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(distance_matrix)